In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/bedroom/bedroom_1.jpg
/kaggle/input/config1/config.json


In [2]:
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.5 MB/s eta 0:00:00a 0:00:01


In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from PIL import Image
import torch
from torchvision import transforms
import json
import matplotlib.pyplot as plt

def load_pipeline(config_path):
    with open(config_path, "r") as f:
        config = json.load(f)

    controlnet_model = ControlNetModel(
        in_channels=config["in_channels"],
        conditioning_channels=config["conditioning_channels"],
        block_out_channels=config["block_out_channels"],
        layers_per_block=config["layers_per_block"],
        down_block_types=config["down_block_types"],
        mid_block_type=config["mid_block_type"],
        upcast_attention=config["upcast_attention"],
        norm_num_groups=config["norm_num_groups"],
        cross_attention_dim=config["cross_attention_dim"],
        attention_head_dim=config["attention_head_dim"],
        conditioning_embedding_out_channels=config["conditioning_embedding_out_channels"]
    ).half()

    pipeline = StableDiffusionControlNetPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",  
        controlnet=controlnet_model,
        torch_dtype=torch.float16  
    ).to('cuda')

    for param in pipeline.unet.parameters():
        param.data = param.data.half()
        if param.grad is not None:
            param.grad.data = param.grad.data.half()

    return pipeline

def generate_design(pipeline, input_image, prompt):
    """
    Pipeline for image generation.
    """
    preprocess = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
    ])
    input_image_tensor = preprocess(input_image).unsqueeze(0).to(pipeline.device, dtype=torch.float16)

    input_image_resized = transforms.Resize((512, 512))(input_image)
    input_image_resized.save("input_image.jpg")
    print("Входное изображение сохранено как input_image.jpg. Проверьте в файловой системе.")

    print(f"Input dtype: {input_image_tensor.dtype}, device: {input_image_tensor.device}")
    print(f"Pipeline UNet dtype: {pipeline.unet.dtype}")
    print(f"Pipeline device: {pipeline.device}")

    output = pipeline(prompt, image=input_image_tensor)
    output_image = output.images[0]

    output_image.save("output_image.jpg")
    print("Output image saved as output_image.jpg")

    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    axs[0].imshow(input_image_resized)
    axs[0].set_title("Input image")
    axs[0].axis("off")
    axs[1].imshow(output_image)
    axs[1].set_title("Output image")
    axs[1].axis("off")
    plt.show()

    return output_image

if __name__ == "__main__":
    config_path = "/kaggle/input/config1/config.json"

    # Загружаем пайплайн
    pipeline = load_pipeline(config_path)

    # Тестовые данные
    input_image = Image.open("/kaggle/input/bedroom/bedroom_1.jpg")
    input_text = "A luxurious living room with modern furniture and a cozy atmosphere"

    # Генерация изображения
    try:
        generate_design(pipeline, input_image, input_text)
    except RuntimeError as e:
        print(f"Ошибка: {e}")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]